In [5]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import geopandas as gpd

# process all the data for "the_year" and save yearly master data  

In [6]:
the_year = 2020

In [9]:
separate_data_folder = '../../data/point_data/data_segments/'
collated_data_folder = '../../data/point_data/'

# get different data modalities: controls, outcomes and enviornmental

In [10]:
controls = pd.read_csv(separate_data_folder + 'controls.csv')
# replace quotes in column names
controls.columns = controls.columns.str.replace('"', '')
controls.columns = controls.columns.str.replace(':', '')
controls.columns = controls.columns.str.replace(',', '')
controls.rename(columns={c:"c_"+c for c in controls.columns if not c in ['geography code', 'geography']}, inplace=True)

controls = controls.drop_duplicates()


len(controls), len(controls.drop_duplicates()), len(set(controls.index)), len(controls['geography code'].unique())

(35672, 35672, 35672, 35672)

In [11]:
outcomes = pd.read_csv(separate_data_folder + '{}_outcomes.csv'.format(the_year))
outcomes.rename(columns={c:"o_"+c for c in outcomes.columns if not c in ['geography code', 'geography']}, inplace=True)

outcomes = outcomes.drop_duplicates()

len(outcomes), len(outcomes.drop_duplicates()), len(set(outcomes.index)), len(outcomes['geography code'].unique())

(32833, 32833, 32833, 32833)

In [12]:
env = pd.read_csv(separate_data_folder + '{}_environment.csv'.format(the_year)).rename(columns={'LSOA21CD':'geography code'})
del env['LSOA21NM']
env.rename(columns={c:"e_"+c for c in env.columns if not c in ['geography code', 'geography']}, inplace=True)


env = env.drop_duplicates()

len(env), len(env.drop_duplicates()), len(set(env.index)), len(env['geography code'].unique())

(37183, 37183, 37183, 33804)

# Merge the different modalities

In [33]:
data = controls.merge(outcomes, on=['geography code'], how='outer')
data = data.merge(env, on=['geography code'], how='outer')

In [34]:
len(data), len(data.drop_duplicates()), len(set(data.index)), len(data['geography code'].unique())

(40085, 40085, 40085, 36706)

### remove duplicates

We have some duplicates because the environmental code produces scores for some LSOAs twice since we process LSOAs in batches based on the region, and some LSOAs fall into two different regions. These rows were not dropped by drop_duplicates that we ran above simply because of the small-number differences between some environmental scores, stemming from the fact that for calculating some of them, we randomly subsample a subset of all the yearly satellite products. 

In [35]:
# group the dataframe by 'geography code' and select the first element from each group
data = data.groupby('geography code').first()
data = data.reset_index().rename(columns={'index': 'geography code'})
len(data), len(data.drop_duplicates()), len(set(data.index)), len(data['geography code'].unique())

(36706, 36706, 36706, 36706)

In [36]:
data.tail()

,geography code,c_percent asian,c_percent black,c_percent mixed,c_percent white,c_percent christian,c_percent jewish,c_percent no religion,c_percent muslim,c_percent no central heating,...,e_surface_runoff_sum,e_water,e_trees,e_grass,e_flooded_vegetation,e_crops,e_shrub_and_scrub,e_built,e_bare,e_snow_and_ice
36701,W01002036,0.030284,0.014175,0.045747,0.900773,0.376372,0.000646,0.531311,0.020658,0.005188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36702,W01002037,0.021596,0.009390,0.029108,0.932394,0.418386,0.002814,0.483114,0.022514,0.012411,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36703,W01002038,0.007115,0.003162,0.022134,0.966008,0.438291,0.000791,0.509494,0.000791,0.020619,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36704,W01002039,0.011932,0.004545,0.026136,0.956250,0.400568,0.000568,0.538636,0.003977,0.001497,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36705,W01002040,0.033453,0.006691,0.036541,0.920226,0.282553,0.000515,0.638703,0.014411,0.013319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# ADD SPATIAL ELEMENTS INTO THE MASTER

# read Region and LSOA shapefiles

In [37]:
regions = gpd.read_file('../../data/auxiliary_data/regions_2022/Regions_(December_2022)_EN_BFC/Regions_(December_2022)_EN_BFC.shp')
lsoas = gpd.read_file('../../data/auxiliary_data/lsoas_2021/LSOA_(Dec_2021)_Boundaries_Generalised_Clipped_EW_(BGC)/LSOA_(Dec_2021)_Boundaries_Generalised_Clipped_EW_(BGC).shp')

In [38]:
len(lsoas)

35672

# spatial join LSOA REGION

In [39]:
lsoas_regions = gpd.sjoin(lsoas, regions, predicate='within')

In [40]:
len(lsoas_regions['LSOA21CD']), len(set(lsoas_regions['LSOA21CD']))

(30523, 30523)

In [41]:
lsoas_regions_mapping = lsoas_regions[['LSOA21CD', 'RGN22CD', 'RGN22NM', 'LSOA21NM']].drop_duplicates()

## save the mapping

This can be used later, if needed in some analyses so we save it.

In [42]:
# lsoas_regions_mapping.to_csv('../../data/auxiliary_data/lsoas_regions_mapping.csv', index=None)

## add geo columns to the point data. we will create two master files: 

### 1) one as a csv
### 2) second as a geojson

In [43]:
# region_geo_columns  = lsoas_regions[['LSOA21CD', 'RGN22CD', 'RGN22NM', 'LSOA21NM', 'geometry']]
geo_columns  = lsoas[['LSOA21CD', 'LSOA21NM', 'geometry']]

In [44]:
spatial_data = data.merge(geo_columns.\
                rename(columns={'LSOA21CD':'geography code'}), # , 'RGN22NM':'region'
                on='geography code')

In [45]:
# region_spatial_data = data.merge(region_geo_columns.\
#                 rename(columns={'LSOA21CD':'geography code', 'RGN22NM':'region'}),
#                 on='geography code')

In [46]:
spatial_data.shape

(35672, 106)

In [47]:
spatial_data = spatial_data[spatial_data['geography code'].str.startswith('E')]

In [48]:
spatial_data.shape

(33755, 106)

In [49]:
spatial_data = spatial_data.set_index('geography code')
spatial_data.head(10)

,c_percent asian,c_percent black,c_percent mixed,c_percent white,c_percent christian,c_percent jewish,c_percent no religion,c_percent muslim,c_percent no central heating,c_percent wood heating,...,e_trees,e_grass,e_flooded_vegetation,e_crops,e_shrub_and_scrub,e_built,e_bare,e_snow_and_ice,LSOA21NM,geometry
geography code,,,,,,,,,,,,,,,,,,,,,
E01000001,0.107191,0.007463,0.037992,0.801221,0.345763,0.019661,0.483390,0.021695,0.026190,0.000000,...,0.053142,0.034637,0.037382,0.029789,0.061613,0.487398,0.086469,0.092108,City of London 001A,"POLYGON ((532105.312 182010.574, 532162.491 18..."
E01000002,0.130592,0.007937,0.043290,0.782107,0.338150,0.025289,0.491329,0.018786,0.019417,0.001214,...,0.047079,0.032114,0.036503,0.028673,0.062030,0.478821,0.094367,0.108474,City of London 001B,"POLYGON ((532634.497 181926.016, 532619.141 18..."
E01000003,0.094913,0.034739,0.062655,0.741315,0.340360,0.026658,0.479851,0.030998,0.035329,0.000000,...,0.057794,0.034344,0.035054,0.029642,0.057333,0.541734,0.070082,0.088695,City of London 001C,"POLYGON ((532135.138 182198.131, 532158.250 18..."
E01000005,0.321526,0.108084,0.071753,0.385104,0.367514,0.012704,0.221416,0.309437,0.012526,0.000000,...,0.044154,0.030236,0.035305,0.028209,0.061742,0.525229,0.109265,0.072893,City of London 001E,"POLYGON ((533808.018 180767.774, 533649.037 18..."
E01000006,0.479675,0.108401,0.037940,0.327913,0.371614,0.000542,0.070423,0.391116,0.027076,0.000000,...,0.066284,0.035672,0.033845,0.033294,0.060597,0.598694,0.072455,0.032890,Barking and Dagenham 016A,"POLYGON ((545122.049 184314.931, 545271.849 18..."
E01000007,0.319683,0.284928,0.062973,0.288025,0.381493,0.000344,0.125559,0.377365,0.031040,0.000000,...,0.066161,0.034978,0.034301,0.032276,0.064485,0.542067,0.087362,0.050718,Barking and Dagenham 015A,"POLYGON ((544180.347 184700.582, 544317.196 18..."
E01000008,0.256125,0.398664,0.048441,0.250000,0.471604,0.000000,0.100223,0.346882,0.027372,0.000000,...,0.102022,0.041083,0.035331,0.038354,0.052078,0.587635,0.046533,0.034572,Barking and Dagenham 015B,"MULTIPOLYGON (((543741.001 184566.000, 543806...."
E01000009,0.489751,0.111911,0.029363,0.303601,0.305987,0.001109,0.092572,0.394124,0.028333,0.000000,...,0.062609,0.032248,0.033476,0.038322,0.066256,0.545876,0.089955,0.030667,Barking and Dagenham 016B,"POLYGON ((544499.837 184704.366, 544630.292 18..."
E01000011,0.548122,0.100939,0.024061,0.283451,0.330394,0.000588,0.054674,0.479130,0.038055,0.000000,...,0.121090,0.049614,0.033997,0.050135,0.071378,0.458662,0.084793,0.034769,Barking and Dagenham 016C,"POLYGON ((544523.402 184789.039, 544667.055 18..."


In [50]:
# region_spatial_data = region_spatial_data.set_index('geography code')
# region_spatial_data.head()

In [50]:
# we see that we lose > 2K rows, i.e., LSOAs if we want to include region
len(spatial_data) #, len(region_spatial_data)

33755

# add cenotroid point for each LSOA

In [51]:
spatial_data = gpd.GeoDataFrame(spatial_data, geometry='geometry')

spatial_data = spatial_data.to_crs(lsoas.crs)
spatial_data['center_coordinates'] = spatial_data['geometry'].centroid

spatial_data["centroid_x"] = spatial_data["center_coordinates"].apply(lambda p: p.x)
spatial_data["centroid_y"] = spatial_data["center_coordinates"].apply(lambda p: p.y)

del spatial_data['center_coordinates']

In [52]:
# region_spatial_data = gpd.GeoDataFrame(region_spatial_data, geometry='geometry')
# region_spatial_data = region_spatial_data.to_crs(lsoas.crs)
# region_spatial_data['center_coordinates'] = region_spatial_data['geometry'].centroid

# region_spatial_data["centroid_x"] = region_spatial_data["center_coordinates"].apply(lambda p: p.x)
# region_spatial_data["centroid_y"] = region_spatial_data["center_coordinates"].apply(lambda p: p.y)

In [53]:
print(spatial_data.shape) #, print(region_spatial_data.shape)

(33755, 107)


## save .csv master file

In [54]:
socdem_columns = [c for c in spatial_data.columns if c.startswith("c_")]
env_columns = [c for c in spatial_data.columns if c.startswith("e_")]
output_columns = [c for c in spatial_data.columns if c.startswith("o_")]
geo_columns = ["LSOA21NM", "geometry"]
geo_columns = geo_columns +[c for c in spatial_data.columns if c.startswith("centroid_")]

columns_order = geo_columns + socdem_columns + env_columns + output_columns

spatial_data = spatial_data[columns_order]

In [55]:
spatial_data.to_csv(collated_data_folder + '{}_spatial_raw_master.csv'.format(the_year))

In [56]:
# region_spatial_data.to_csv('../../data/point_data/{}_region_spatial_raw_master.csv'.format(the_year))

## save .geojson master file

In [57]:
# here, we keep LSOA shape the main geometry; but we also save LSOA cenotroid points saved. this is useful for some othet types of models, such as spatial regression
spatial_data_lsoa = spatial_data.copy()
# del spatial_data_lsoa['center_coordinates']
# spatial_data_lsoa['center_coordinates'] = spatial_data_lsoa['center_coordinates'].astype(str)
spatial_data_lsoa.to_file(collated_data_folder + '{}_spatial_raw_master.geojson'.format(the_year), driver='GeoJSON')


# # here, we keep LSOA centroid points as the main geometry; but we also save polygon geometries of LSOAs as lsoa_geometry. the reason is that for spatiall fold validation we need point coordinates for geometry
# spatial_data.rename(columns={'geometry':'lsoa_geometry', 'center_coordinates':'geometry'}, inplace=True)
# # spatial_data['center_coordinates'] = spatial_data['center_coordinates'].astype(str)
# spatial_data['lsoa_geometry'] = spatial_data['lsoa_geometry'].astype(str)
# spatial_data.to_file('../../data/point_data/{}_spatial_raw_master.geojson'.format(the_year), driver='GeoJSON')

In [58]:
# for col in spatial_data.columns:
#     if spatial_data[col].dtype != 'float64':
#         print(col, spatial_data[col].dtype)

In [60]:
# region_spatial_data.rename(columns={'geometry':'lsoa_geometry', 'center_coordinates':'geometry'}, inplace=True)

# # spatial_data['center_coordinates'] = spatial_data['center_coordinates'].astype(str)
# region_spatial_data['lsoa_geometry'] = region_spatial_data['lsoa_geometry'].astype(str)

# region_spatial_data.to_file('../../data/point_data/{}_region_spatial_raw_master.geojson'.format(the_year), driver='GeoJSON')